# Segmenting and Clustering Neighborhoods in Toronto

### Import Libraries

In [94]:
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [38]:
import pandas as pd
import requests
!conda install -c anaconda beautifulsoup4 --yes #it can take several minutes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.



### Create table

In [36]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody

columns = ['PostalCode', 'Borough', 'Neighbourhood']
data = dict({key:[]*len(columns) for key in columns})

for row in table.find_all('tr'):
    for i,column in zip(row.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

df = pd.DataFrame.from_dict(data=data)[columns]
print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove rows where Borough is 'Not assigned' / Merge / Reset Index

+ drop(df[df.Borough == "Not assigned"].index) : drop rows with value "Not assigned" in column "Borough"
+ sort_values(by=['PostalCode']): group by Postal Code and merge the same Neighbourhood
+ reset_index(drop=True) :  Reset the index to be sequential after the rows' drop


In [37]:
df2 = df.drop(df[df.Borough == "Not assigned"].index).sort_values(by=['PostalCode']).reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:
df2.shape

(103, 3)

## Add Coordinates

In [39]:
df_c = pd.read_csv("http://cocl.us/Geospatial_data")
df_c.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


+ Change the "Postal Code" column name in Coordinates df to the same of Neighbourhood: "PostalCode" so we can merge both

In [52]:
df_c.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_c.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


+ Left merge

In [54]:
df3 = df2.merge(df_c, on=['PostalCode'], how='left')
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


+ Check for any null value

In [55]:
df3.isnull().any().any()

False

In [61]:
df3.shape

(103, 5)

## Explore and cluster the neighborhoods in Toronto

In [57]:
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#### Find Toronto Coordinates

In [60]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torinto-ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Torinto-ON are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

+ filter the Data Frame with only boroughs that contain the word Toronto

In [72]:
neighborhoods = df3[df3.Borough.str.contains("Toronto")]

In [75]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

+ Define Foursquare Credentials and Version

In [77]:
# @hidden_cell
#
CLIENT_ID = 'FQLROFUGKHHND5KQG5H0KSJ5JCTFVIMQPUZ4L1JOEXPKT1BR' # your Foursquare ID
CLIENT_SECRET = 'FJIL1OWA5ILV1A0DMPW2FZKL0XC0SYIGHOFXCWO5PEFHV4KF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Explore Neighborhoods in Manhattan

#### This function will get the top 100 venues that are within a radius of 500 meters for each Neighborhood

In [80]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Let's run the function to create de df

In [81]:
toronto_venues = getNearbyVenues(names=neighborhoods['Borough'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
Downtown Toronto
East Toronto


#### Let's check the size of the resulting dataframe

In [82]:
print(toronto_venues.shape)
toronto_venues.head()

(1608, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,East Toronto,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


Let's check how many venues were returned for each neighborhood

In [83]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,115,115,115,115,115,115
Downtown Toronto,1213,1213,1213,1213,1213,1213
East Toronto,122,122,122,122,122,122
West Toronto,158,158,158,158,158,158


#### Let's find out how many unique categories can be curated from all the returned venues

In [84]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## Analyze Each Neighborhood

In [85]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [86]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017391,0.000000,...,0.000000,0.008696,0.008696,0.000000,0.008696,0.000000,0.008696,0.000000,0.000000,0.008696
1,Downtown Toronto,0.000824,0.000824,0.000824,0.000824,0.001649,0.002473,0.001649,0.014015,0.001649,...,0.000824,0.000000,0.000824,0.002473,0.010717,0.001649,0.003298,0.004946,0.001649,0.005771
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590,0.000000,...,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.008197,0.000000,0.016393
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.018987,0.000000,0.012658,0.006329,0.000000,0.012658


#### Let's print each neighborhood along with the top 5 most common venues

In [87]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3            Park  0.04
4     Pizza Place  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2           Restaurant  0.04
3                Hotel  0.03
4  Japanese Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2                Café  0.04
3             Brewery  0.04
4  Italian Restaurant  0.04


----West Toronto----
                venue  freq
0                Café  0.07
1                 Bar  0.06
2          Restaurant  0.04
3         Coffee Shop  0.04
4  Italian Restaurant  0.04




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [88]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each borough.

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
3,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Bookstore,Breakfast Spot,Park


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters. (n_samples=4 should be >= n_clusters=5)

In [98]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [102]:
# add clustering labels
#boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub


Finally, let's visualize the resulting clusters

In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
45,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
46,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
47,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
48,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
49,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
63,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
64,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub
65,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pub


In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
51,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
52,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
53,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
54,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
55,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
56,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
57,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
58,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
59,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant


In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
41,East Toronto,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
42,East Toronto,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
43,East Toronto,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant
87,East Toronto,2,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Ice Cream Shop,Park,Pub,Pizza Place,Restaurant


# the end